In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json
from itertools import chain

In [50]:
class AirlineQualityScraper:
    def __init__(self, url):
        self.url = url
        self.download_page()
        self.info_list = ['Type Of Traveller', 'Seat Type','Route','Date Flown']
        self.ratings = ['Seat Comfort', 'Cabin Staff Service', 'Food & Beverages', 'Inflight Entertainment', 'Ground Service', 'Wifi & Connectivity', 'Value For Money']

    def download_page(self):
        # method for downloading the webpage
        self.page = requests.get(self.url).text
    
    def rating_info_search(self, html_table, topic, content_type='td'):
        locate = html_table.find('td', text=topic)
        if locate is not None:
            value = locate.findNext(content_type).text
        else:
            value = None
        return value
    
    def count_star(self, html_table, topic):
        star = html_table.find('td', text=topic)
        count = 0
        if star is not None:
            for i in range(5):
                star = star.findNext('span')
                if "star fill" in str(star):
                    count += 1
        return count

    def scrape_data(self):
        soup = BeautifulSoup(self.page, "html.parser")
        data = {'title':[],'user':[],'user_type':[],'review':[],'publish_date':[]}
        
        reviews = soup.find_all("div",{"class": "body"})
        for review in reviews:
            title = review.find(class_="text_header").text
            data['title'].append(title.replace('"',''))
            user_name = review.find("span",{"itemprop":"name"}).text
            data['user'].append(user_name)
            publish_date = review.find("time",{'itemprop':"datePublished"}).text
            data['publish_date'].append(publish_date)
            
            review_raw = review.find('div',{'itemprop':"reviewBody"}).text
            if '|' in review_raw:
                review_split = review_raw.split('|')
                data['user_type'].append(''.join(ch for ch in review_split[0] if ch.isalnum()))
                data['review'].append(review_split[1])
            else:
                data['user_type'].append('Not Verified')
                data['review'].append(review_raw)
            
            review_table = review.find('table', {'class': 'review-ratings'})
            for topic in self.info_list:
                topic_value = self.rating_info_search(review_table,topic)
                if topic not in data.keys():
                    data[topic] = [topic_value]
                else:
                    data[topic].append(topic_value)
            for rating_item in self.ratings:
                rating = self.count_star(review_table,rating_item)
                if rating_item not in data.keys():
                    data[rating_item] = [rating]
                else:
                    data[rating_item].append(rating)

        return data



#for url in urls:

#    x = TripHotelScraper(url)

#    print(x.scrape_data())

In [51]:
aircanada_review = "https://www.airlinequality.com/airline-reviews/air-canada"
#x = AirlineQualityScraper(aircanada_review)
#scrapped = x.scrape_data()

In [52]:
def scrape_pages(url,n_pages):
    print('Scrapping page:', 1)
    page_1 = AirlineQualityScraper(url)
    scrapped = page_1.scrape_data()
    scrapped['page'] = [1]*len(scrapped['user'])
    for i in range(2,n_pages+1): 
        print('Scrapping page:', i)
        page = AirlineQualityScraper(url+'/page/'+str(i)+'/')
        data_scrapped = page.scrape_data()
        data_scrapped['page'] = [i]*len(data_scrapped['user'])
        for k, v in data_scrapped.items():
            scrapped[k] += data_scrapped[k]
    return scrapped

In [53]:
data_air_canada = scrape_pages(aircanada_review,165)

Scrapping page: 1
Scrapping page: 2
Scrapping page: 3
Scrapping page: 4
Scrapping page: 5
Scrapping page: 6
Scrapping page: 7
Scrapping page: 8
Scrapping page: 9
Scrapping page: 10
Scrapping page: 11
Scrapping page: 12
Scrapping page: 13
Scrapping page: 14
Scrapping page: 15
Scrapping page: 16
Scrapping page: 17
Scrapping page: 18
Scrapping page: 19
Scrapping page: 20
Scrapping page: 21
Scrapping page: 22
Scrapping page: 23
Scrapping page: 24
Scrapping page: 25
Scrapping page: 26
Scrapping page: 27
Scrapping page: 28
Scrapping page: 29
Scrapping page: 30
Scrapping page: 31
Scrapping page: 32
Scrapping page: 33
Scrapping page: 34
Scrapping page: 35
Scrapping page: 36
Scrapping page: 37
Scrapping page: 38
Scrapping page: 39
Scrapping page: 40
Scrapping page: 41
Scrapping page: 42
Scrapping page: 43
Scrapping page: 44
Scrapping page: 45
Scrapping page: 46
Scrapping page: 47
Scrapping page: 48
Scrapping page: 49
Scrapping page: 50
Scrapping page: 51
Scrapping page: 52
Scrapping page: 53
Sc

In [58]:
aircanada_review = pd.DataFrame(data_air_canada)
aircanada_review.to_csv('AirCanada_review_raw.csv')

In [ ]:
#Remove stop words
stemmer = PorterStemmer()
stop_words = stopwords.words('english')
lst_of_tokenized = comments['comment_tokenized']
for sentence in lst_of_tokenized:
    new_sentence = []
    for s in sentence:
        if s in string.punctuation or s == '..':
            sentence.remove(s)
        if s in stop_words:
            sentence.remove(s)